In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/aditya/git/kaggle_protein
%env PROJECT_PATH = /home/aditya/git/kaggle_protein

/home/aditya/git/kaggle_protein
env: PROJECT_PATH=/home/aditya/git/kaggle_protein


In [3]:
from model.dataset import *
from model.arch import *
from model.criterion import *
from model.train import *

In [4]:
from utils.envs import *
from utils.common import *
from utils.interface import *

In [5]:
import pandas as pd
import torch
import torchvision
import torchsummary
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [6]:
random_state = 1
device = torch.device('cuda')

In [7]:
train_df = pd.read_csv(train_repo_path)
dev_df, val_df = split_dev_val(train_df, val_size = 0.20, random_state = random_state)

In [8]:
train_dataset = ProteinTrainDataset(train_df, train_path, device)
dev_dataset = ProteinTrainDataset(dev_df, train_path, device)
val_dataset = ProteinTrainDataset(val_df, train_path, device)
test_dataset = ProteinTestDataset(test_path, device)

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
dev_dataloader = DataLoader(dev_dataset, batch_size = 32, shuffle = True)
val_dataloader = DataLoader(val_dataset, batch_size = 1, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle = True)

In [10]:
preload_model = torchvision.models.resnet50(pretrained=True).to(device)
header_model = Header(512, 0.2).to(device)
model = Net(preload_model, header_model, device).to(device)

In [11]:
n_epoch = 5
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss().to(device)

In [12]:
n_obs, batch_size, n_batch_per_epoch = get_batch_info(dev_dataloader)

In [13]:
def loss_fn(model, criterion, data):
    img, target = data
    prediction = model(img)
    loss = criterion(prediction, target)
    return loss

In [14]:
def metric_fn(model, data):
    img, target = data
    prediction = model(img)
    metric = multi_accuracy(prediction, target)
    return metric

In [15]:
def pred_fn(model, data):
    img = data
    prediction = model(img)
    return prediction.data.cpu().numpy().reshape(-1).tolist()

In [16]:
model = fit_model(model, n_epoch, dev_dataloader, optimizer, criterion, loss_fn, metric_fn, val_dataloader, checkpoint = True, model_fn = 'first')

/home/aditya/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/aditya/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


 val_loss : 0.1427378609328443, val_metric : 0.9503850367408746


FileNotFoundError: [Errno 2] No such file or directory: '/home/aditya/git/kaggle_protein/output/model_checkpoint/first_0_model.pth'